In [2]:
import os
import gc
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models
from sklearn.metrics import classification_report, f1_score
import numpy as np
from tqdm import tqdm
import pandas as pd

# =====================================================
# CONFIGURATION
# =====================================================
DATA_DIR = "./data/processed"
SAVE_DIR = "./resnet-18"
os.makedirs(SAVE_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

EPOCHS = 20
PATIENCE = 4
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
TRAIN_LIMIT = 10000
TEST_LIMIT = 2000
VAL_SPLIT = 0.1  # 10% for validation

# =====================================================
# HELPER FUNCTIONS
# =====================================================

def load_dataset(dataset_name, split_type, aug_type):
    """Load dataset from pickle file."""
    path = os.path.join(DATA_DIR, f"{dataset_name}_{split_type}", f"{aug_type}.pkl")
    print(f"Loading: {path}")
    with open(path, "rb") as f:
        data = pickle.load(f)
    return data["images"], data["labels"]

def preprocess_images(images, dataset_type):
    """Convert images to tensor format and normalize."""
    # Ensure 4D: (N, H, W, C)
    if images.ndim == 3:
        images = np.expand_dims(images, axis=-1)
    
    # Transpose to PyTorch format: (N, C, H, W)
    images = images.transpose(0, 3, 1, 2)
    
    # Convert to tensor and normalize to [0, 1]
    tensor = torch.tensor(images, dtype=torch.float32)
    if tensor.max() > 1.0:
        tensor = tensor / 255.0
    
    # Convert grayscale to RGB by repeating channels
    if dataset_type == "grayscale" and tensor.shape[1] == 1:
        tensor = tensor.repeat(1, 3, 1, 1)
    
    return tensor

def get_model(num_classes, pretrained=True):
    """Initialize ResNet-18 model."""
    if pretrained:
        model = models.resnet18(weights="IMAGENET1K_V1")
    else:
        model = models.resnet18(weights=None)
    
    # Replace final layer for our number of classes
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(device)

def train_model(model, train_loader, val_loader, epochs=EPOCHS, patience=PATIENCE):
    """Train model with early stopping."""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    best_val_f1 = 0
    patience_counter = 0
    best_state = None
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for X, y in pbar:
            X, y = X.to(device), y.to(device)
            
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_total += y.size(0)
            train_correct += (predicted == y).sum().item()
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_loss = running_loss / len(train_loader)
        train_acc = 100 * train_correct / train_total
        
        # Validation phase
        val_f1, val_acc = evaluate_model(model, val_loader, return_accuracy=True)
        
        print(f"Epoch {epoch+1}/{epochs} - "
              f"Loss: {avg_loss:.4f}, "
              f"Train Acc: {train_acc:.2f}%, "
              f"Val Acc: {val_acc:.2f}%, "
              f"Val F1: {val_f1:.4f}")
        
        # Early stopping check
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            patience_counter = 0
            best_state = model.state_dict().copy()
            print(f"  ✓ New best validation F1: {best_val_f1:.4f}")
        else:
            patience_counter += 1
            print(f"  No improvement ({patience_counter}/{patience})")
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break
    
    # Load best model
    if best_state is not None:
        model.load_state_dict(best_state)
        print(f"\nLoaded best model with Val F1: {best_val_f1:.4f}")
    
    return model, best_val_f1

def evaluate_model(model, data_loader, return_accuracy=False, silent=False):
    """Evaluate model on given data loader."""
    model.eval()
    preds, true = [], []
    
    with torch.no_grad():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            true.extend(y.cpu().numpy())
    
    f1 = f1_score(true, preds, average="macro")
    
    if return_accuracy:
        acc = 100 * np.mean(np.array(true) == np.array(preds))
        return f1, acc
    
    if not silent:
        report = classification_report(true, preds, digits=4)
        print(report)
        return f1, true, preds
    
    return f1

def clear_memory():
    """Clear GPU and CPU memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# =====================================================
# MAIN EXECUTION
# =====================================================

datasets = {
    "mnist": {"classes": 10, "type": "grayscale"},
    "cifar10": {"classes": 10, "type": "color"}
}

train_types = ["original", "mixed_augmented", "combined_augmented"]
test_types = ["original", "all_combined", "noise", "occlusion_25", "rotation_15", "scaling_0.8"]

results = []

for dataset_name, info in datasets.items():
    print(f"\n{'='*80}")
    print(f"PROCESSING DATASET: {dataset_name.upper()}")
    print(f"{'='*80}\n")
    
    for train_aug in train_types:
        print(f"\n{'*'*60}")
        print(f"Training configuration: {train_aug}")
        print(f"{'*'*60}\n")
        
        # Load training data ONCE
        train_images, train_labels = load_dataset(dataset_name, "train", train_aug)
        train_images = train_images[:TRAIN_LIMIT]
        train_labels = train_labels[:TRAIN_LIMIT]
        
        print(f"Training samples: {len(train_images)}")
        print(f"Image shape: {train_images.shape}")
        print(f"Label distribution: {np.bincount(train_labels)}")
        
        # Preprocess training data
        X_train = preprocess_images(train_images, info["type"])
        y_train = torch.tensor(train_labels, dtype=torch.long)
        
        # Create train/val split
        full_dataset = TensorDataset(X_train, y_train)
        train_size = int((1 - VAL_SPLIT) * len(full_dataset))
        val_size = len(full_dataset) - train_size
        
        # Use fixed seed for reproducibility
        torch.manual_seed(42)
        train_subset, val_subset = torch.utils.data.random_split(
            full_dataset, [train_size, val_size]
        )
        
        train_loader = DataLoader(
            train_subset, 
            batch_size=BATCH_SIZE, 
            shuffle=True,
            num_workers=0,
            pin_memory=True if torch.cuda.is_available() else False
        )
        val_loader = DataLoader(
            val_subset, 
            batch_size=BATCH_SIZE,
            num_workers=0,
            pin_memory=True if torch.cuda.is_available() else False
        )
        
        print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}\n")
        
        # Train model ONCE per training configuration
        model = get_model(info["classes"])
        model, best_val_f1 = train_model(model, train_loader, val_loader)
        
        # Evaluate on all test sets
        for test_aug in test_types:
            print(f"\n{'─'*60}")
            print(f"Evaluating on test set: {test_aug}")
            print(f"{'─'*60}")
            
            # Load test data
            test_images, test_labels = load_dataset(dataset_name, "test", test_aug)
            test_images = test_images[:TEST_LIMIT]
            test_labels = test_labels[:TEST_LIMIT]
            
            print(f"Test samples: {len(test_images)}")
            
            # Preprocess test data
            X_test = preprocess_images(test_images, info["type"])
            y_test = torch.tensor(test_labels, dtype=torch.long)
            
            test_loader = DataLoader(
                TensorDataset(X_test, y_test), 
                batch_size=BATCH_SIZE,
                num_workers=0,
                pin_memory=True if torch.cuda.is_available() else False
            )
            
            # Evaluate
            test_f1, y_true, y_pred = evaluate_model(model, test_loader)
            test_acc = 100 * np.mean(np.array(y_true) == np.array(y_pred))
            
            print(f"\nTest Accuracy: {test_acc:.2f}%")
            print(f"Test F1 Score: {test_f1:.4f}")
            
            # Save detailed report
            report = classification_report(y_true, y_pred, digits=4)
            report_path = os.path.join(
                SAVE_DIR, 
                f"{dataset_name}_train-{train_aug}_test-{test_aug}.txt"
            )
            with open(report_path, "w") as f:
                f.write(f"Dataset: {dataset_name}\n")
                f.write(f"Training Type: {train_aug}\n")
                f.write(f"Test Type: {test_aug}\n")
                f.write(f"{'='*60}\n\n")
                f.write(report)
                f.write(f"\n{'='*60}\n")
                f.write(f"Best Validation F1: {best_val_f1:.4f}\n")
                f.write(f"Test F1: {test_f1:.4f}\n")
                f.write(f"Test Accuracy: {test_acc:.2f}%\n")
            
            # Store results
            results.append({
                "dataset": dataset_name,
                "train_type": train_aug,
                "test_type": test_aug,
                "best_val_f1": best_val_f1,
                "test_f1": test_f1,
                "test_accuracy": test_acc
            })
            
            # Clean up test data
            del X_test, y_test, test_loader, test_images, test_labels
            clear_memory()
        
        # Clean up model and training data after all test evaluations
        del model, X_train, y_train, train_loader, val_loader, train_images, train_labels
        del train_subset, val_subset, full_dataset
        clear_memory()
        
        print(f"\n✓ Completed training configuration: {train_aug}\n")


Using device: cuda

PROCESSING DATASET: MNIST


************************************************************
Training configuration: original
************************************************************

Loading: ./data/processed\mnist_train\original.pkl
Training samples: 10000
Image shape: (10000, 128, 128, 1)
Label distribution: [1001 1127  991 1032  980  863 1014 1070  944  978]
Train batches: 141, Val batches: 16



Epoch 1/20: 100%|██████████| 141/141 [00:04<00:00, 32.41it/s, loss=0.0972]


Epoch 1/20 - Loss: 0.1761, Train Acc: 95.23%, Val Acc: 98.30%, Val F1: 0.9829
  ✓ New best validation F1: 0.9829


Epoch 2/20: 100%|██████████| 141/141 [00:04<00:00, 34.94it/s, loss=0.0025]


Epoch 2/20 - Loss: 0.0210, Train Acc: 99.53%, Val Acc: 99.00%, Val F1: 0.9896
  ✓ New best validation F1: 0.9896


...

Epoch 15/20 - Loss: 0.4109, Train Acc: 86.22%, Val Acc: 69.70%, Val F1: 0.6921
  No improvement (2/4)


Epoch 16/20: 100%|██████████| 141/141 [00:10<00:00, 13.64it/s, loss=0.0450]


Epoch 16/20 - Loss: 0.1141, Train Acc: 96.29%, Val Acc: 75.30%, Val F1: 0.7436
  No improvement (3/4)


Epoch 17/20: 100%|██████████| 141/141 [00:09<00:00, 15.51it/s, loss=0.0057]


Epoch 17/20 - Loss: 0.0225, Train Acc: 99.46%, Val Acc: 75.20%, Val F1: 0.7509
  No improvement (4/4)
Early stopping triggered at epoch 17

Loaded best model with Val F1: 0.7754

────────────────────────────────────────────────────────────
Evaluating on test set: original
────────────────────────────────────────────────────────────
Loading: ./data/processed\cifar10_test\original.pkl
Test samples: 2000
              precision    recall  f1-score   support

           0     0.4272    0.8980    0.5789       196
           1     0.6915    0.9848    0.8125       198
           2     0.7710    0.5179    0.6196       195
           3     0.6000    0.5729    0.5861       199
           4     0.7917    0.5758    0.6667       198
           5     0.5703    0.7676    0.6544       185
           6     0.8168    0.7639    0.7895       216
           7     0.8862    0.5648    0.6899       193
           8     0.9000    0.4562    0.6055       217
           9     0.8726    0.6749    0.7611       203


In [3]:

# =====================================================
# SAVE SUMMARY
# =====================================================
df = pd.DataFrame(results)
csv_path = os.path.join(SAVE_DIR, "resnet18_results.csv")
df.to_csv(csv_path, index=False)

print(f"\n{'='*80}")
print("EXPERIMENT SUMMARY")
print(f"{'='*80}\n")
print(df.to_string(index=False))
print(f"\n✓ All experiments complete!")
print(f"✓ Summary saved to: {csv_path}")
print(f"✓ Individual reports saved to: {SAVE_DIR}/")


EXPERIMENT SUMMARY

dataset         train_type    test_type  best_val_f1  test_f1  test_accuracy
  mnist           original     original     0.991041 0.989169          98.95
  mnist           original all_combined     0.991041 0.367146          40.15
  mnist           original        noise     0.991041 0.091232          16.55
  mnist           original occlusion_25     0.991041 0.922393          92.35
  mnist           original  rotation_15     0.991041 0.975389          97.55
  mnist           original  scaling_0.8     0.991041 0.983435          98.35
  mnist    mixed_augmented     original     0.990962 0.985232          98.50
  mnist    mixed_augmented all_combined     0.990962 0.354786          39.85
  mnist    mixed_augmented        noise     0.990962 0.138574          26.45
  mnist    mixed_augmented occlusion_25     0.990962 0.891756          89.10
  mnist    mixed_augmented  rotation_15     0.990962 0.946534          94.60
  mnist    mixed_augmented  scaling_0.8     0.990962 0.